In [1]:
!pip install boto3

You are using pip version 19.0.3, however version 21.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
!pip install pyspark

You are using pip version 19.0.3, however version 21.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
import urllib
import boto3
import sys
import os
import pandas as pd
import csv
import io
import numpy as np

In [2]:
ACCESS_KEY = "#################"
SECRET_KEY = "#################"
ENCODED_SECRET_KEY = urllib.parse.quote(SECRET_KEY, "")
AWS_BUCKET_NAME = "cs-516-project-flight-data"
MOUNT_NAME = "s3data"

In [3]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [4]:
spark = SparkSession.builder.getOrCreate()

C:\Users\Advait\Anaconda3\lib\site-packages\pyspark\context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [5]:
s3_client =boto3.client('s3')
s3_bucket_name= AWS_BUCKET_NAME
s3 = boto3.resource('s3',
                    aws_access_key_id= ACCESS_KEY,
                    aws_secret_access_key= SECRET_KEY)

C:\Users\Advait\Anaconda3\lib\site-packages\boto3\compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [6]:
my_bucket=s3.Bucket(s3_bucket_name)
bucket_list = []
for file in my_bucket.objects.filter():
    file_name=file.key
    if file_name.find(".csv")!=-1:
        bucket_list.append(file.key)
length_bucket_list=print(len(bucket_list))
print(bucket_list)

3
['airlines.csv', 'airports.csv', 'flights.csv']


In [7]:
spark.sparkContext._jsc.hadoopConfiguration().set('fs.s3a.access.key', ACCESS_KEY)
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.secret.key", SECRET_KEY)
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3.amazonaws.com")

In [27]:
airlines = spark.read.csv("s3a://cs-516-project-flight-data/" + bucket_list[0], header = True)
airports = spark.read.csv("s3a://cs-516-project-flight-data/" + bucket_list[1], header = True)
flights = spark.read.csv("s3a://cs-516-project-flight-data/" + bucket_list[2], header = True)

In [28]:
airlines.show()
airports.show()
flights.show()

+---------+--------------------+
|IATA_CODE|             AIRLINE|
+---------+--------------------+
|       UA|United Air Lines ...|
|       AA|American Airlines...|
|       US|     US Airways Inc.|
|       F9|Frontier Airlines...|
|       B6|     JetBlue Airways|
|       OO|Skywest Airlines ...|
|       AS|Alaska Airlines Inc.|
|       NK|    Spirit Air Lines|
|       WN|Southwest Airline...|
|       DL|Delta Air Lines Inc.|
|       EV|Atlantic Southeas...|
|       HA|Hawaiian Airlines...|
|       MQ|American Eagle Ai...|
|       VX|      Virgin America|
+---------+--------------------+

+---------+--------------------+-------------+-----+-------+--------+----------+
|IATA_CODE|             AIRPORT|         CITY|STATE|COUNTRY|LATITUDE| LONGITUDE|
+---------+--------------------+-------------+-----+-------+--------+----------+
|      ABE|Lehigh Valley Int...|    Allentown|   PA|    USA|40.65236| -75.44040|
|      ABI|Abilene Regional ...|      Abilene|   TX|    USA|32.41132| -99.68190|


In [17]:
import os

In [29]:
print((flights.count(), len(flights.columns)))

(5819079, 31)


In [30]:
flights.createOrReplaceTempView('flights')

In [31]:
print(spark.catalog.listTables())

[Table(name='flights', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]


In [33]:
query = "SELECT AIRLINE, FLIGHT_NUMBER, TAIL_NUMBER, ORIGIN_AIRPORT, DESTINATION_AIRPORT, SCHEDULED_DEPARTURE FROM flights LIMIT 5"
flights5 = spark.sql(query)
flights5.show()

+-------+-------------+-----------+--------------+-------------------+-------------------+
|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|
+-------+-------------+-----------+--------------+-------------------+-------------------+
|     AS|           98|     N407AS|           ANC|                SEA|               0005|
|     AA|         2336|     N3KUAA|           LAX|                PBI|               0010|
|     US|          840|     N171US|           SFO|                CLT|               0020|
|     AA|          258|     N3HYAA|           LAX|                MIA|               0020|
|     AS|          135|     N527AS|           SEA|                ANC|               0025|
+-------+-------------+-----------+--------------+-------------------+-------------------+



In [34]:
# calculating duration of flight in hrs
flights = spark.table("flights")
flights = flights.withColumn('duration_hrs', flights.AIR_TIME/60.)
flights.select('duration_hrs').show(10)

+------------------+
|      duration_hrs|
+------------------+
| 2.816666666666667|
| 4.383333333333334|
| 4.433333333333334|
|               4.3|
| 3.316666666666667|
| 3.433333333333333|
| 2.566666666666667|
|               3.8|
|2.8833333333333333|
|               3.1|
+------------------+
only showing top 10 rows



In [35]:
flights = flights.withColumn("MONTH", flights.MONTH.cast("integer"))
flights = flights.withColumn("DAY_OF_WEEK", flights.DAY_OF_WEEK.cast("integer"))
flights = flights.withColumn("AIR_TIME", flights.AIR_TIME.cast("integer"))
flights = flights.withColumn("DISTANCE", flights.DISTANCE.cast("double"))
flights = flights.withColumn("ARRIVAL_DELAY", flights.ARRIVAL_DELAY.cast("integer"))

In [36]:
flights.filter(flights.ORIGIN_AIRPORT == 'PDX').groupBy().min('DISTANCE').show()
flights.filter(flights.ORIGIN_AIRPORT == 'SEA').groupBy().max('AIR_TIME').show()

+-------------+
|min(DISTANCE)|
+-------------+
|        129.0|
+-------------+

+-------------+
|max(AIR_TIME)|
+-------------+
|          412|
+-------------+



In [37]:
by_plane = flights.groupBy("TAIL_NUMBER")
by_plane.count().show(10)

+-----------+-----+
|TAIL_NUMBER|count|
+-----------+-----+
|     N38451|  946|
|     N567AA| 1458|
|     N623NK| 1669|
|     N442AS| 1210|
|     N902DE| 1704|
|     N4YUAA| 1290|
|     N466SW| 1942|
|     N516UA|  439|
|     N866AS| 2397|
|     N499AA| 1292|
+-----------+-----+
only showing top 10 rows



In [38]:
airports = airports.withColumnRenamed("IATA_CODE", "DESTINATION_AIRPORT")
flights_with_airports = flights.join(airports , on = 'DESTINATION_AIRPORT', how = 'leftouter')

In [40]:
flights_with_airports.select('FLIGHT_NUMBER', 'AIRPORT', 'CITY', 'STATE', 'COUNTRY', 'LATITUDE', 'LONGITUDE').show(10)

+-------------+--------------------+-----------------+-----+-------+--------+----------+
|FLIGHT_NUMBER|             AIRPORT|             CITY|STATE|COUNTRY|LATITUDE| LONGITUDE|
+-------------+--------------------+-----------------+-----+-------+--------+----------+
|           98|Seattle-Tacoma In...|          Seattle|   WA|    USA|47.44898|-122.30931|
|         2336|Palm Beach Intern...|  West Palm Beach|   FL|    USA|26.68316| -80.09559|
|          840|Charlotte Douglas...|        Charlotte|   NC|    USA|35.21401| -80.94313|
|          258|Miami Internation...|            Miami|   FL|    USA|25.79325| -80.29056|
|          135|Ted Stevens Ancho...|        Anchorage|   AK|    USA|61.17432|-149.99619|
|          806|Minneapolis-Saint...|      Minneapolis|   MN|    USA|44.88055| -93.21692|
|          612|Minneapolis-Saint...|      Minneapolis|   MN|    USA|44.88055| -93.21692|
|         2013|Charlotte Douglas...|        Charlotte|   NC|    USA|35.21401| -80.94313|
|         1112|Dallas

In [42]:
model_data = flights.select('MONTH', 'DAY_OF_WEEK', 'AIRLINE', 'TAIL_NUMBER', 'DESTINATION_AIRPORT', 'AIR_TIME', 'DISTANCE', 'ARRIVAL_DELAY',)
model_data = model_data.filter("ARRIVAL_DELAY is not NULL and AIRLINE is not NULL and AIR_TIME is not NULL and TAIL_NUMBER is not NULL")

In [43]:
model_data = model_data.withColumn("is_late", model_data.ARRIVAL_DELAY > 0)
model_data = model_data.withColumn("is_late", model_data.is_late.cast("integer"))
model_data = model_data.withColumnRenamed("is_late", 'label')

In [44]:
model_data.show(15)

+-----+-----------+-------+-----------+-------------------+--------+--------+-------------+-----+
|MONTH|DAY_OF_WEEK|AIRLINE|TAIL_NUMBER|DESTINATION_AIRPORT|AIR_TIME|DISTANCE|ARRIVAL_DELAY|label|
+-----+-----------+-------+-----------+-------------------+--------+--------+-------------+-----+
|    1|          4|     AS|     N407AS|                SEA|     169|  1448.0|          -22|    0|
|    1|          4|     AA|     N3KUAA|                PBI|     263|  2330.0|           -9|    0|
|    1|          4|     US|     N171US|                CLT|     266|  2296.0|            5|    1|
|    1|          4|     AA|     N3HYAA|                MIA|     258|  2342.0|           -9|    0|
|    1|          4|     AS|     N527AS|                ANC|     199|  1448.0|          -21|    0|
|    1|          4|     DL|     N3730B|                MSP|     206|  1589.0|            8|    1|
|    1|          4|     NK|     N635NK|                MSP|     154|  1299.0|          -17|    0|
|    1|          4| 

In [46]:
print('Flights distrubution:')
model_data.groupBy('label').count().show()

Flights distrubution:
+-----+-------+
|label|  count|
+-----+-------+
|    1|2086896|
|    0|3627112|
+-----+-------+



In [47]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [48]:
airline_indexer = StringIndexer(inputCol="AIRLINE", outputCol="airline_index")
airline_encoder = OneHotEncoder(inputCol="airline_index", outputCol="airline_fact")

In [49]:
dest_indexer = StringIndexer(inputCol="DESTINATION_AIRPORT", outputCol="dest_index")
dest_encoder = OneHotEncoder(inputCol="dest_index", outputCol="dest_fact")

In [50]:
tail_indexer = StringIndexer(inputCol="TAIL_NUMBER", outputCol="tail_index")
tail_encoder = OneHotEncoder(inputCol="tail_index", outputCol="tail_fact")

In [51]:
from pyspark.ml.feature import VectorAssembler
vec_assembler = VectorAssembler(inputCols=["MONTH", "DAY_OF_WEEK", "AIR_TIME", "DISTANCE", "airline_fact", "dest_fact", "tail_fact"], outputCol="features")

In [52]:
# Create Pipeline
from pyspark.ml import Pipeline
flights_pipe = Pipeline(stages=[dest_indexer, dest_encoder, airline_indexer, airline_encoder, tail_indexer, tail_encoder, vec_assembler])

In [53]:
piped_data = flights_pipe.fit(model_data).transform(model_data)

In [54]:
train_data, test_data = piped_data.randomSplit([.7, .3])

In [55]:
# Create the model
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression()

In [56]:
import pyspark.ml.evaluation as evals
evaluator = evals.BinaryClassificationEvaluator(metricName="areaUnderROC")

In [57]:
best_lr = lr.fit(train_data)
print(best_lr)

LogisticRegressionModel: uid=LogisticRegression_589426bd013a, numClasses=2, numFeatures=5540


In [68]:
# Use the model to predict from the test data
test_results = best_lr.transform(test_data)

In [67]:
test_results.select("MONTH","DAY_OF_WEEK","AIRLINE","TAIL_NUMBER","DESTINATION_AIRPORT","DISTANCE","label").show(10)

+-----+-----------+-------+-----------+-------------------+--------+-----+
|MONTH|DAY_OF_WEEK|AIRLINE|TAIL_NUMBER|DESTINATION_AIRPORT|DISTANCE|label|
+-----+-----------+-------+-----------+-------------------+--------+-----+
|    1|          1|     AA|     N001AA|                LAX|   748.0|    1|
|    1|          1|     AA|     N002AA|                ATL|   594.0|    0|
|    1|          1|     AA|     N002AA|                ATL|   594.0|    1|
|    1|          1|     AA|     N002AA|                ATL|   731.0|    0|
|    1|          1|     AA|     N003AA|                ATL|   594.0|    0|
|    1|          1|     AA|     N003AA|                DFW|  1047.0|    0|
|    1|          1|     AA|     N003AA|                MIA|   964.0|    1|
|    1|          1|     AA|     N004AA|                DFW|   551.0|    1|
|    1|          1|     AA|     N004AA|                DFW|   731.0|    0|
|    1|          1|     AA|     N004AA|                DFW|   733.0|    1|
+-----+-----------+------

In [69]:
print(evaluator.evaluate(test_results))

0.6268305105549999
